In [4]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from transliterate import translit
import os
import numpy as np
import joblib
from typing import Dict
import yaml
import json
import warnings

warnings.filterwarnings("ignore")

In [5]:
config_path = '../config/params.yml'
config = yaml.load(open(config_path), Loader=yaml.FullLoader)

preproc = config['preprocessing']
training = config['train']
evaluate = config['evaluate']

# check columns with train
column_sequence_path = preproc['unique_values_path']
with open(column_sequence_path, encoding="utf8") as json_file:
    column_sequence = json.load(json_file)

In [6]:
data = pd.read_json(evaluate['predict_path'])

In [7]:
data

,room_count,district,area,street,house,metro,time_metro,facilities,floor,attic,square,transport_type
0,1,ЦАО,Якиманка,ул. Большая Полянка,28к1,Полянка,1,"[Можно с детьми, Ванна, Стиральная машина, Пос...",11,14,38,пешком
1,2,СВАО,Отрадное,ул. Декабристов,21,Отрадное,4,"[Можно с животными, Можно с детьми, Стиральная...",2,12,45,пешком
2,3,ВАО,Сокольники,ул. Матросская Тишина,23/7С3,Сокольники,16,"[Можно с детьми, Ванна, Стиральная машина, Пос...",5,5,80,пешком
3,2,ЦАО,Мещанский,Рождественский бул.,17,Трубная,5,"[Можно с животными, Стиральная машина, Посудом...",4,5,77,пешком
4,2,ЦАО,Пресненский,1-й Красногвардейский проезд,22с2,Тестовская,4,"[Можно с животными, Можно с детьми, Ванна, Сти...",48,78,73,пешком
...,...,...,...,...,...,...,...,...,...,...,...,...
4873,2,ЗАО,Фили-Давыдково,Кременчугская ул.,44К2,Давыдково,8,"[Можно с детьми, Стиральная машина, Мебель на ...",1,5,38,пешком
4874,1,САО,Сокол,Песчаная ул.,13,Сокол,6,"[Можно с животными, Можно с детьми, Ванна, Сти...",10,14,40,пешком
4875,3,ЦАО,Красносельский,Скорняжный пер.,7к2,Красные ворота,8,"[Можно с детьми, Можно с животными, Ванна]",4,16,69,пешком
4876,2,ЮЗАО,Ясенево,Вильнюсская ул.,8К2,Ясенево,13,"[Можно с детьми, Ванна, Стиральная машина, Меб...",2,16,54,пешком


In [26]:
def check_columns_evaluate(data: pd.DataFrame,
                           unique_values_path: str) -> pd.DataFrame:
    """
    Проверка на наличие признаков из train и упорядочивание признаков согласно train
    :param data: датасет test
    :param unique_values_path: путь до списока с признаками train для сравнения
    :return: датасет test
    """
    with open(unique_values_path, 'r', encoding='utf8') as json_file:
        unique_values = json.load(json_file)

    column_sequence = unique_values.keys()

    assert set(column_sequence) == set(data.columns), "Разные признаки"
    return data[column_sequence]

def save_unique_train_data(
    data: pd.DataFrame, drop_columns: list, target_column: str, unique_values_path: str
) -> None:
    """
    Сохранение словаря с признаками и уникальными значениями
    :param drop_columns: список с признаками для удаления
    :param data: датасет
    :param target_column: целевая переменная
    :param unique_values_path: путь до файла со словарем
    :return: None
    """
    unique_df = data.drop(
        columns=drop_columns + [target_column], axis=1, errors="ignore"
    )
    # создаем словарь с уникальными значениями для вывода в UI
    dict_unique = {
        column: unique_df[column].explode().unique().tolist() if isinstance(
            unique_df[column].iloc[0], list) else
        unique_df[column].unique().tolist()
        for column in unique_df.columns
    }
    with open(unique_values_path, "w", encoding="utf-8") as file:
        json.dump(dict_unique, file, ensure_ascii=False)

In [27]:
def pipeline_preprocess(data: pd.DataFrame,
                        flg_evaluate: bool = True,
                        **kwargs):
    """
    Обрабатывает исходные данные, применяя различные преобразования и заполнение отсутствующих значений.

    Аргументы:
    - data: pandas.DataFrame, исходные данные

    Возвращает:
    - pandas.DataFrame, обработанные данные
    """
    data = data.drop(kwargs["drop_columns"], axis=1, errors="ignore")
    # проверка dataset на совпадение с признаками из train
    # либо сохранение уникальных данных с признаками из train
    if flg_evaluate:
        data = check_columns_evaluate(
            data=data, unique_values_path=kwargs["unique_values_path"])
    else:
        save_unique_train_data(
            data=data,
            drop_columns=kwargs["drop_columns"],
            target_column=kwargs["target_column"],
            unique_values_path=kwargs["unique_values_path"],
        )

    data = data.drop(kwargs["drop_columns"], axis=1, errors="ignore")

    data = data.fillna({
        col: -1000 if data[col].dtype in ['int64', 'float64'] else 'unknown'
        for col in data.columns
    })

    # преобразование столбцов со списками в ячейках в бинаризованные
    list_columns = [
        col for col in data.columns if isinstance(data[col].iloc[0], list)
    ]
    if len(list_columns) > 0:
        mlb = MultiLabelBinarizer()
        list_columns = [
            col for col in data.columns if isinstance(data[col].iloc[0], list)
        ]
        data_bin = pd.DataFrame(mlb.fit_transform(data[list_columns[0]]),
                                columns=mlb.classes_,
                                index=data.index)
        data = pd.concat([data.drop(list_columns, axis=1), data_bin], axis=1)

    translit_columns = {
        column: translit(column.replace(" ", "_").replace("'", ""),
                         'ru',
                         reversed=True).replace("'", "")
        for column in data.columns
    }
    data = data.rename(columns=translit_columns)

    data = transform_types(data=data,
                           change_type_columns=kwargs["change_type_columns"])

    dict_category = {
        key: "category"
        for key in data.select_dtypes(["object"]).columns
    }
    data = transform_types(data=data, change_type_columns=dict_category)

    return data

In [34]:
data_proc_test = pipeline_preprocess(data=da, **preproc)

AttributeError: 'dict' object has no attribute 'drop'

In [29]:
data_proc_test.columns

Index(['room_count', 'district', 'area', 'metro', 'time_metro', 'floor',
       'attic', 'square', 'transport_type', 'Vanna', 'Dushevaja_kabina',
       'Internet', 'Konditsioner', 'Mebel_v_komnatah', 'Mebel_na_kuhne',
       'Mozhno_s_detmi', 'Mozhno_s_zhivotnymi', 'Posudomoechnaja_mashina',
       'Stiralnaja_mashina', 'Televizor', 'Holodilnik'],
      dtype='object')

In [5]:
da={
  "room_count": 3,
  "district": "ЦАО",
  "area": "Пресненский",
  "metro": "Шелепиха",
  "time_metro": 5,
  "floor": 15,
  "attic": 36,
  "square": 65,
  "transport_type": "пешком",
  "Vanna": 0,
  "Dushevaja_kabina": 0,
  "Internet": 0,
  "Konditsioner": 0,
  "Mebel_v_komnatah": 0,
  "Mebel_na_kuhne": 0,
  "Mozhno_s_detmi": 0,
  "Mozhno_s_zhivotnymi": 0,
  "Posudomoechnaja_mashina": 0,
  "Stiralnaja_mashina": 0,
  "Televizor": 0,
  "Holodilnik": 0
}

In [31]:
column_names = data_proc_test.columns.tolist()
dict_keys = list(da.keys())

if column_names == dict_keys:
    print("Названия столбцов DataFrame и ключи словаря совпадают")
else:
    print("Названия столбцов DataFrame и ключи словаря не совпадают")

Названия столбцов DataFrame и ключи словаря совпадают


In [32]:
model = joblib.load(training['model_path'])
data_proc_test['predict'] = model.predict(data_proc_test)

In [33]:
data_proc_test

,room_count,district,area,metro,time_metro,floor,attic,square,transport_type,Vanna,...,Konditsioner,Mebel_v_komnatah,Mebel_na_kuhne,Mozhno_s_detmi,Mozhno_s_zhivotnymi,Posudomoechnaja_mashina,Stiralnaja_mashina,Televizor,Holodilnik,predict
0,1,ЦАО,Якиманка,Полянка,1,11,14,38,пешком,1,...,1,1,1,1,0,1,1,1,1,76010.569467
1,2,СВАО,Отрадное,Отрадное,4,2,12,45,пешком,0,...,0,1,1,1,1,0,1,0,1,43830.879180
2,3,ВАО,Сокольники,Сокольники,16,5,5,80,пешком,1,...,1,1,1,1,0,1,1,1,1,84652.847718
3,2,ЦАО,Мещанский,Трубная,5,4,5,77,пешком,0,...,1,1,1,0,1,1,1,1,1,144232.708393
4,2,ЦАО,Пресненский,Тестовская,4,48,78,73,пешком,1,...,1,1,1,1,1,1,1,1,1,218006.133643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4873,2,ЗАО,Фили-Давыдково,Давыдково,8,1,5,38,пешком,0,...,0,1,1,1,0,0,1,1,1,34359.869209
4874,1,САО,Сокол,Сокол,6,10,14,40,пешком,1,...,1,1,1,1,1,0,1,1,1,43988.269060
4875,3,ЦАО,Красносельский,Красные ворота,8,4,16,69,пешком,1,...,0,0,0,1,1,0,0,0,0,90829.549953
4876,2,ЮЗАО,Ясенево,Ясенево,13,2,16,54,пешком,1,...,0,1,1,1,0,0,1,1,1,43599.894234


In [1]:
from pydantic import BaseModel

In [2]:
class FlatParams(BaseModel):
    """
    Признаки для получения результатов модели
    """
    room_count: int
    district: str
    area: str
    metro: str
    time_metro: int
    floor: int
    attic: int
    square: int
    transport_type: str
    Vanna:int
    Dushevaja_kabina: int
    Internet: int
    Konditsioner: int
    Mebel_v_komnatah: int
    Mebel_na_kuhne: int
    Mozhno_s_detmi: int
    Mozhno_s_zhivotnymi: int
    Posudomoechnaja_mashina: int
    Stiralnaja_mashina: int
    Televizor: int
    Holodilnik: int

In [3]:
def prediction_input(flat: FlatParams):
    """
    Предсказание модели по введенным данным
    """
    features = [
        [
            flat.room_count,
            flat.district,
            flat.area,
            flat.metro,
            flat.time_metro,
            flat.floor,
            flat.attic,
            flat.square,
            flat.transport_type,
            flat.Vanna,
            flat.Dushevaja_kabina,
            flat.Internet,
            flat.Konditsioner,
            flat.Mebel_v_komnatah,
            flat.Mebel_na_kuhne,
            flat.Mozhno_s_detmi,
            flat.Mozhno_s_zhivotnymi,
            flat.Posudomoechnaja_mashina,
            flat.Stiralnaja_mashina,
            flat.Televizor,
            flat.Holodilnik
        ]
    ]

    cols = [
        'room_count',
        'district',
        'area',
        'metro',
        'time_metro',
        'floor',
        'attic',
        'square',
        'transport_type',
        'Vanna',
        'Dushevaja_kabina',
        'Internet',
        'Konditsioner',
        'Mebel_v_komnatah',
        'Mebel_na_kuhne',
        'Mozhno_s_detmi',
        'Mozhno_s_zhivotnymi',
        'Posudomoechnaja_mashina',
        'Stiralnaja_mashina',
        'Televizor',
        'Holodilnik'
    ]

    data = pd.DataFrame(features, columns=cols)

In [6]:
data=prediction_input(da)

AttributeError: 'dict' object has no attribute 'room_count'